## R-UMLDA implementation w/ Tensorly
---
### The R-UMLDA Objective

A set of tensor objects $ \{X_{1},X_{2},\ldots,X_{M}\} $ is availalble for training, where each sample $ X_{i}\in\mathbb{R}^{I_{1}xI_{2}x\cdots xI_{N}},i=1,2,\ldots,M $ and there exits $c$ classes. The R-UMLDA objective is to project each sample using **$ P $ EMPs** so that

* Fisher's discrimination criterion (FDC) for each of the p EMPs is maximized ($ \frac{S_{B}}{S_{W}} $)
* the coordinate vectors/projected features ($ g_{p}\in\mathbb{R}^{M},p=1,2,\ldots,P $) are all uncorellated.

In short, for the p-th EMP (data is centered):
$$ u_{p}^{(1)},u_{p}^{(2)},\ldots,u_{p}^{(N)}=\arg\max_{u_{p}^{(1)},u_{p}^{(2)},\ldots,u_{p}^{(N)}}\frac{S_{B}}{S_{W}}=\frac{\sum_{c=1}^{C}M_{c}\overline{y_{c_{p}}}^{2}}{\sum_{m=1}^{M}(y_{m_{p}}-\overline{y_{c_{mp}}})^{2}}  $$
subject to:
$$ \frac{g_{p}^{T}g_{q}}{||g_{p}||||g_{q}||}=δ_{pq}=\begin{cases}
1 & p=q\\
0 & otherwise
\end{cases},\,p,q=1,\ldots,P $$
where: 

$ M_{c} $ denotes the number of samples beloning in class $c$,

$ y_{c_{p}} $ denotes the mean of the projected feature of samples belonging in $c$ using the p-th EMP,

$ \overline{y_{c_{mp}}} $ denotes the mean of the projected features of samples belonging to the same class $c$ as the m-th sample using the p-th EMP and

$ y_{m_{p}} $ denotes the (scalar) projection of the m-th sample using the p-th EMP.

Moreover, we will employ regularizaton in order to:
* Counter the effects of the SSS scenario and
* Improve the generalization capabilities

Just as before, we cannot solve directly for all of the N projection vectors of an EMP. Assuming we are solving for the n-th mode projetion vector of the p-th EMP, the optimization problem becomes:

$$ u_{p}^{(n)}=\arg\max_{u_{p}^{(n)}}\frac{u_{p}^{(n)^{T}}S_{B}^{(n)}u_{p}^{(n)}}{u_{p}^{(n)^{T}}S_{W}^{(n)}u_{p}^{(n)}}=\frac{(u_{p}^{(n)^{T}})\sum_{c=1}^{C}M_{c}\overline{y_{c_{p}}^{(n)}}\overline{y_{c_{p}}^{(n)^{T}}}u_{p}^{(n)}}{(u_{p}^{(n)^{T}})(\sum_{m=1}^{M}(y_{m_{p}}^{(n)}-\overline{y_{c_{mp}}^{(n)}})(y_{m_{p}}^{(n)}-\overline{y_{c_{mp}}^{(n)}})^{T}+γλ_{max}(S_{w}^{(n)})I_{I_{n}})(u_{p}^{(n)})} $$
subject to:
$$ u_{p}^{(n)^{T}}Y_{p}^{(n)}g_{q}=0\, q=1,2,\ldots,p-1 $$


where:

$ (n) $ denotes partial projection to all modes except the n-th,

$ S_{w}^{(n)}=\sum_{m=1}^{M}(X_{m(n)}-X_{c_{m}(n)})(X_{m(n)}-X_{c_{m}(n)})^{T} $ denotes the within-class scatter for the n-mode vectors of the training samples,

$ γ $ is the regularization hyperparameter,

$ I_{I_{n}} $ is the $I_{n}xI_{n} $ identity matrix,

$ Y_{p}^{(n)}=\left[\begin{array}{cccc}
y_{1p}^{(n)} & y_{2p}^{(n)} & \cdots & y_{Mp}^{(n)}\end{array}\right]\in\mathbb{R}^{I_{n}xM} $ (partial projection of all samples using all projection vectors but the n-th) and

$ g_{p} $ denotes the p-th coordinate vector ($ g_{p}=Y_{p}^{(n)^{T}}u_{p}^{(n)} $).

Solution: Eq. 11 [2]

### Algorithm

**Input:**  A set of tensor objects $ \{X_{1},X_{2},\ldots,X_{M}\} $, $ X_{i}\in\mathbb{R}^{I_{1}xI_{2}x\cdots xI_{N}},i=1,2,\ldots,M $

**Output:** The TVP $ u_{p}^{(n)},p=1,2,\ldots,P,n=1,2,\ldots,N $ that separate classes as best as possible according to the FDC,  while producing uncorellated features.

    1. Center the data
    2. Initialize Projection matrices
    3. Estimate largest eigenvalue of n-mode within class scatter
    4. UMLDA Loop
        for p=1:P
            for k=1:K
                for n=1:N
                    4a. Calculate partial projection
                    4b. Calculate Rp, BCS, WCS
                    4c. Set U(n) to be the eigenvector respective to the largest eigenvalue of WCS^(-1) @ Rp @ BCS
---
#### References
[1] Multilinear Subspace Learning: Dimensionality Reduction of Multidimensional Data, Haiping Lu, K. N. Plataniotis, and A. N. Venetsanopoulos, Chapman & Hall/CRC Press Machine Learning and Pattern Recognition Series, Taylor and Francis, ISBN: 978-1-4398572-4-3, 2013.

[2] Haiping Lu, K.N. Plataniotis, and A.N. Venetsanopoulos, Uncorrelated Multilinear Discriminant Analysis with Regularization and Aggregation for Tensor Object Recognition", IEEE Transactions on Neural Networks, Vol. 20, No. 1, Page: 103-123, Jan. 2009.

In [18]:
# Import necessary libraries

import tensorly as tl
from scipy.io import loadmat
import scipy
import numpy as np
import copy


In [19]:
# Import and reshape dataset

FERETC80A45S6 = tl.tensor(loadmat('FERETC80A45S6/FERETC80A45S6_32x32.mat')['fea'],dtype=np.double)
FERETC80A45S6_labels = tl.tensor(loadmat('FERETC80A45S6/FERETC80A45S6_32x32.mat')['gnd'],dtype=np.double)

print(f'Initial shape: {FERETC80A45S6.shape}')

# Transpose to make sure the number of samples is the last dim!

FERETC80A45S6 = copy.deepcopy(np.transpose(FERETC80A45S6))
FERETC80A45S6 = copy.deepcopy(FERETC80A45S6.reshape(32,32,FERETC80A45S6.shape[-1]))

for i in range(FERETC80A45S6.shape[-1]): 
    FERETC80A45S6[:,:,i] = copy.deepcopy(np.transpose(FERETC80A45S6[:,:,i]))

print(f'After reshaping: {FERETC80A45S6.shape}')

# Load train-test subset

trainIdx = tl.tensor(loadmat('FERETC80A45S6/4Train/1.mat')['trainIdx'])
testIdx = tl.tensor(loadmat('FERETC80A45S6/4Train/1.mat')['testIdx'])

# squeeze to skips dims with 1 component (e.g. (32, 32, 320, 1)->(32, 32, 320))

train_set = np.squeeze(FERETC80A45S6[:,:,trainIdx-1],-1) 
test_set = np.squeeze(FERETC80A45S6[:,:,testIdx-1],-1)

gnd_train = np.squeeze(FERETC80A45S6_labels[trainIdx-1])
gnd_test = np.squeeze(FERETC80A45S6_labels[testIdx-1])

print(f'Train set shape: {train_set.shape}')
print(f'Test set shape: {test_set.shape}')


Initial shape: (1145, 1024)
After reshaping: (32, 32, 1145)
Train set shape: (32, 32, 320)
Test set shape: (32, 32, 825)


In [14]:
class myRUMLDA():
    '''
    An object-oriented implementation of R-UMLDA (Uncorrelated Multilinear Discriminant Analysis w/ Regularization).
    
    Methods
    -------
    __init__(): Initialzies the myRUMLDA object and handles error-checking on parameters the user has entered.
    est_largest_l_n_mode_wcs__(): Mainly for internal use. Estimate largest eigenvalue of 
    n-mode within class scatter.
    fit(): Performs UMLDA with given arguments.
    transform(): Returns the projected data.
    
    Attributes
    ----------
    self.maxK: No of iterations (user-defined)
    self.numP: The number of EMPs per TVP (user-defined)
    self.gamma: Regularization parameter (user-defined)
    self.sample_modes: # of modes of given dataset
    self.sample_dims: # of modes of a sample
    self.samples_no: # of samples
    self.no_of_classes: # of class labels
    self.samples_per_class: (numpy array) contains # of elemets per class
    self.projection_matrices: 2D List (NxP) containing the projection vectors
    self.projected_data: Coordinate vector/Projected data, shape: (no_of_samples,P)
    '''
    
    def __init__(self,numP,maxK,gamma):
        '''
        numP: The number of EMPs in the TVP.
        self.maxK: No of iterations (user-defined)
        self.gamma: Regularization parameter (user-defined)
        '''

        # Error handling on input        
        
        try:
            
            if int(maxK) <= 0:
                
                raise Exception('Max iters (k) has to be a positive integer.')
                
            self.maxK = maxK
                
        except:

            raise Exception('Max iters (k) has to be a positive integer.')
            
        try:
            
            if int(numP) <= 0:
                
                raise Exception('No of features (numP) has to be a positive integer.')
                
            self.numP = numP
                
        except:

            raise Exception('No of features (numP) has to be a positive integer.')
        
        try: # Can gamma be negative?
                            
            self.gamma = float(gamma)
                
        except:

            raise Exception('Gamma (reg parameter) has to be a float.')
            
    def est_largest_l_n_mode_wcs_(self,samples,samples_labels):
        '''
        Estimate largest eigenvalue of n-mode within class scatter.
        Returns a list of eigenvalues for each mode.
        '''
        
        largest_eig_per_mode = []
        
        for n in range(self.sample_dims):
            
            within_class_scatter = np.zeros((samples.shape[n],samples.shape[n]))
            
#             between_class_scatter = np.zeros((samples.shape[n],samples.shape[n]))
            
            for c in range(1,self.no_of_classes+1):
                
                # Find all samples with given class label
                
                indexes = np.where(samples_labels == c)
                               
                samples_of_class = samples[...,indexes]
                
                # Remove dims with 1 component

                samples_of_class = np.squeeze(samples_of_class)
                
                # Find mean sample of class
                
                class_mean = tl.mean(samples_of_class,axis=-1)
                
                # Mode-n unfolding
                
#                 XDiff = tl.unfold(class_mean,n)
                
                # These samples contribute to the Between Class Scatter...
                
#                 between_class_scatter = between_class_scatter + len(indexes[0]) * XDiff @ np.transpose(XDiff)
                
                # ... and of course the Within Class Scatter
                
                for j in range(len(indexes[0])): # Eq. (9) [2]
                    
                    XDiff = tl.unfold(samples[...,indexes[0][j]],n) - tl.unfold(class_mean,n)

                    within_class_scatter = within_class_scatter + XDiff @ np.transpose(XDiff)
                
            eigenvals, eigenvecs = np.linalg.eig(within_class_scatter)

            # eig doesnt not necessarily return sorted eigenvalues, we need to sort them

            indexOrd = np.argsort(eigenvals)[::-1] 

            eigenvals_sorted = eigenvals[indexOrd]
            
            largest_eig_per_mode.append(eigenvals_sorted[0])
                
        return largest_eig_per_mode
        
    def fit(self,samples,samples_labels):
        '''
        Performs UMLDA with given arguments.
        samples: Input tensor of (N+1) dimensions (+1: Group input in one larger tensor)
        samples_labels: Labels of given samples.
        '''
        
        # Gather and store basic info on the train data
    
        self.sample_modes = len(samples.shape)
    
        self.sample_dims = len(samples.shape) - 1

        self.samples_no = samples.shape[-1]
        
        self.no_of_classes = len(np.unique(samples_labels))
        
        self.samples_per_class = np.zeros((self.no_of_classes,1))
  
        for i in range(self.samples_no): # Remember: no of samples in class 80 are in self.samples_per_class[79] !
            
            self.samples_per_class[int(samples_labels[i])-1][0] = self.samples_per_class[int(samples_labels[i])-1][0] + 1
        
        ########################################################
        # 1. Center the data
        ########################################################
        
        # Find mean tensor
        
        self.samples_mean = np.mean(samples,axis=self.sample_dims)

        # Center the data

        for i in range(self.samples_no):

            samples[...,i] = np.subtract(samples[...,i],self.samples_mean)

        ########################################################
        # 2. Estimate largest eigenvalue of n-mode within class scatter
        ########################################################
        
        # l_e is a list of length equal to the number of sample_dims
        # and contains the largest eigenvalue of the n-mode scatter.
        
        l_e = self.est_largest_l_n_mode_wcs_(samples,samples_labels)
        
        ########################################################
        # 3. Projection matrix initialization
        ########################################################
        
        # Create the list that will hold the projection vectors
        # Dimensions: N x p
        # Initialization method: Uniform
        
        self.projection_matrices = []
        
        for n in range(self.sample_dims):
            
            self.projection_matrices.append([])
            
            for p in range(self.numP):
                
               self.projection_matrices[n].append(np.ones((samples.shape[n],1)) / np.linalg.norm(np.ones((samples.shape[n],1))))
         
        ########################################################
        # 4. UMLDA Loop
        ########################################################
        
        kappa = 10e-3 # This is added during the computation of R in order to get better results pg.12 [2]
        
        for p in range(self.numP): # step p: Caclulate the p-th EMP

            for k in range(self.maxK):
                
                for n in range(self.sample_dims):
                    
                    # Calculate the partial projection
                    
                    projection_modes = [*range(self.sample_dims)]
                    
                    projection_modes.remove(n) # Without using the current mode!
                        
                    projection_matrices2use = np.array(self.projection_matrices)[projection_modes,p]
                    
                    partial_projection = tl.tenalg.multi_mode_dot(samples,projection_matrices2use,modes=projection_modes,transpose=True) #.reshape(samples.shape[n],self.samples_no)              
                    
                    # Remove dims with 1 component
                    
                    if n == 0: partial_projection = np.squeeze(partial_projection,1)
                    else: partial_projection = np.squeeze(partial_projection,0)
                        
                    # Calculate BCS and WCS
                    
                    between_class_scatter = np.zeros((samples.shape[n],samples.shape[n]))
                    within_class_scatter = np.zeros((samples.shape[n],samples.shape[n]))
                    
                    for c in range(1,self.no_of_classes+1):
                        
                        indexes = np.where(samples_labels == c)[0]
                        
                        partial_projection_of_class_samples = partial_projection[:,indexes]
                        
                        class_mean = tl.mean(partial_projection_of_class_samples,axis=-1)
                        
                        # Eq. (7),(8) [2]
                        
                        for m in range(int(self.samples_per_class[c-1])): 
                            
                            YDiff = np.subtract(partial_projection_of_class_samples[:,m],class_mean)
                            
                            YDiff = YDiff[...,np.newaxis] # Add one dim for numpy reasons (e.g (32,) becomes (32,1))
                            
                            within_class_scatter = within_class_scatter + YDiff * np.transpose(YDiff)
                        
                        between_class_scatter = between_class_scatter + self.samples_per_class[c-1] * class_mean[...,np.newaxis] @ np.transpose(class_mean[...,np.newaxis])
                        
                    within_class_scatter = within_class_scatter + self.gamma * l_e[n] * np.eye(samples.shape[n])
                    
                    # Update projection vectors
                    
                    if p > 0:
                        
                        invSW = np.linalg.inv(within_class_scatter)
                        
                        GYSYG = np.linalg.inv(np.transpose(Gps) @ np.transpose(partial_projection) @ invSW @ partial_projection @ Gps + kappa * np.eye(p))
    
                        RSB = np.eye(samples.shape[n]) - partial_projection @ Gps @ GYSYG @ np.transpose(Gps) @ np.transpose(partial_projection) @ invSW
                        
                        between_class_scatter = RSB @ between_class_scatter
                
                        eigenvals, eigenvecs = scipy.linalg.eig(np.linalg.inv(within_class_scatter) @ between_class_scatter)

                        indexOrd = np.argsort(eigenvals)[::-1]

                        eigenvecs_sorted = eigenvecs[:,indexOrd]

                        respective_eigenvector = eigenvecs_sorted[:,0][...,np.newaxis]
            
                    else:
            
                        # pg.7 [2]
    
                        eigenvals, eigenvecs = scipy.linalg.eig(np.linalg.inv(within_class_scatter) @ between_class_scatter)

                        # eig doesnt not necessarily return sorted eigenvalues, we need to sort them

                        indexOrd = np.argsort(eigenvals)[::-1]

                        eigenvecs_sorted = eigenvecs[:,indexOrd]

                        respective_eigenvector = eigenvecs_sorted[:,0][...,np.newaxis]
                        
                    # In order to get consistent results, force the first component of
                    # each eigenvector to be positive

                    if respective_eigenvector[0] < 0.0:

                        respective_eigenvector = respective_eigenvector * (-1)
                        
                    # Normalize and save eigenvector
                    
                    self.projection_matrices[n][p] = np.array(respective_eigenvector / np.linalg.norm(respective_eigenvector),copy=True)
                    
            # Update coordinate/projection vector at p
        
            projection_matrices2use = np.array(self.projection_matrices)[:,p]
            
            gp = tl.tenalg.multi_mode_dot(samples,projection_matrices2use,modes=[*range(self.sample_dims)],transpose=True)
            
            # Remove dims with 1 component
            
            gp = np.transpose(np.squeeze(gp,1))

            if p == 0:
                
                # Gps: Coordinate vectors/projected data, shape: (no_of_samples,P)
        
                Gps = gp
                
            else:

                Gps =  np.append(Gps,gp,axis=1) # Gps is in R^(p)x(M)
               
            # Store projected data to an attribute for easier access

            self.projected_data = Gps
        
    def transform(self):
        '''
        Returns projected data.
        '''

        return self.projected_data


---
## Examples

In [17]:
# Example 1

# Create a RUMLDA object and fit the dataset

rumlda = myRUMLDA(numP=10,maxK=2,gamma=0.01)
rumlda.fit(train_set,gnd_train)

# Print size of projected data

print(f'Initial dataset dimentsions: {train_set.shape}')
print(f'Projected dataset dimensions: {rumlda.projected_data.shape}')

# Print last 5 samples

print('\nLast 5 samples of dataset\n')

for i in range(5,0,-1):
    print(rumlda.projected_data[(-1)*i])


Initial dataset dimentsions: (32, 32, 320)
Projected dataset dimensions: (320, 10)

Last 5 samples of dataset

[-361.92593877 -246.27494986  320.41517831  135.47306974  205.68375144
 -193.6720902    71.90894727 -299.85391768  -29.55092469  -16.02955626]
[ 143.13450757   89.92832496  113.58801227   34.4189721   -39.7045285
   45.70854008  200.32569362 -186.26566352 -279.67030279  224.83590522]
[ 225.14081183   15.62720897   46.58909169 -127.94935978   34.55952992
 -242.1742454   217.60564205 -121.22772416 -234.10596575  168.92257677]
[ 224.20630489  149.03509764  111.75381044   25.68338882 -139.34947863
 -154.52934698   14.6833845  -316.71107775  -27.64050278  133.85848418]
[ 397.31304374  -48.1316686   -52.10242135 -133.93373502  -70.76882787
 -179.61405104 -112.24756417  -99.68463025 -213.65845614  113.33278171]
